# Домашна задача 1, 2
## Обработка на природните јазици 2018/2019
### Андреј Јанчевски  - 151003

In [62]:
from __future__ import print_function
# -*- coding: utf-8 -*-
import re
import numpy as np
from collections import OrderedDict
from functools import cmp_to_key

### Задача 1

In [63]:
def distance(word1, word2, delete_cost=1, insert_cost=1, substitute_cost=2):
    dist = None
    n = len(word1)
    m = len(word2)
    dist_matrix = np.zeros((n + 1, m + 1), dtype="uint8")
    for i in range(1, n + 1):
        dist_matrix[i][0] = dist_matrix[i - 1][0] + delete_cost
    for j in range(1, m + 1):
        dist_matrix[0][j] = dist_matrix[0][j - 1] + insert_cost
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            s = 0
            if word1[i - 1] != word2[j - 1]:
                s = substitute_cost
            dist_matrix[i][j] = min(
                dist_matrix[i - 1][j - 1] + s,
                min(dist_matrix[i - 1][j] + delete_cost,
                    dist_matrix[i][j - 1] + insert_cost))
    dist = dist_matrix[n][m]
    return dist


print(distance("lanugage", "language"))

2


### Задача 2

In [64]:
def distance(word1,
             word2,
             Damerau=False,
             delete_cost=1,
             insert_cost=1,
             substitute_cost=2,
             transpose_cost=1):
    dist = None
    n = len(word1)
    m = len(word2)
    dist_matrix = np.zeros((n + 1, m + 1), dtype="uint8")
    for i in range(1, n + 1):
        dist_matrix[i][0] = dist_matrix[i - 1][0] + delete_cost
    for j in range(1, m + 1):
        dist_matrix[0][j] = dist_matrix[0][j - 1] + insert_cost
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            s = 0
            if word1[i - 1] != word2[j - 1]:
                s = substitute_cost
            t = dist_matrix[i - 1][j - 1] + s
            if Damerau and i > 1 and j > 1 and word1[i - 2] == word2[
                    j - 1] and word1[i - 1] == word2[j - 2]:
                t = dist_matrix[i - 2][j - 2] + transpose_cost
            dist_matrix[i][j] = min(
                min(dist_matrix[i - 1][j - 1] + s, t),
                min(dist_matrix[i - 1][j] + delete_cost,
                    dist_matrix[i][j - 1] + insert_cost))
    dist = dist_matrix[n][m]
    return dist


print(distance("lanugage", "language", Damerau=True))

1


### Задача 3

In [65]:
def distance(word1,
             word2,
             Damerau=False,
             Backtrace=True,
             delete_cost=1,
             insert_cost=1,
             substitute_cost=2,
             transpose_cost=1):
    dist = None
    backtrace = None
    n = len(word1)
    m = len(word2)
    dist_matrix = np.zeros((n + 1, m + 1), dtype="uint8")
    for i in range(1, n + 1):
        dist_matrix[i][0] = dist_matrix[i - 1][0] + delete_cost
    for j in range(1, m + 1):
        dist_matrix[0][j] = dist_matrix[0][j - 1] + insert_cost
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            s = 0
            if word1[i - 1] != word2[j - 1]:
                s = substitute_cost
            t = dist_matrix[i - 1][j - 1] + s
            if Damerau and i > 1 and j > 1 and word1[i - 2] == word2[
                    j - 1] and word1[i - 1] == word2[j - 2]:
                t = dist_matrix[i - 2][j - 2] + transpose_cost
            dist_matrix[i][j] = min(
                min(dist_matrix[i - 1][j - 1] + s, t),
                min(dist_matrix[i - 1][j] + delete_cost,
                    dist_matrix[i][j - 1] + insert_cost))
    dist = dist_matrix[n][m]
    if Backtrace:
        i = n
        j = m
        stack = []
        while i > 0 and j > 0:
            min_distance = dist_matrix[i][j]
            s = 0
            if word1[i - 1] != word2[j - 1]:
                s = substitute_cost
            t = dist_matrix[i - 1][j - 1] + s
            if Damerau and i > 1 and j > 1 and word1[i - 2] == word2[
                    j - 1] and word1[i - 1] == word2[j - 2]:
                t = dist_matrix[i - 2][j - 2] + transpose_cost

            if min_distance == dist_matrix[i - 1][j - 1] + s:
                i -= 1
                j -= 1
                if s == substitute_cost:
                    stack.append("S")
                else:
                    stack.append("M")
            elif min_distance == dist_matrix[i][j - 1] + insert_cost:
                stack.append("I")
                j -= 1
            elif min_distance == dist_matrix[i - 1][j] + delete_cost:
                stack.append("D")
                i -= 1
            elif min_distance == t:
                stack.append("T")
                i -= 2
                j -= 2

        while i > 0:
            stack.append("D")
            i -= 1
        while j > 0:
            stack.append("I")
            j -= 1
        backtrace = ""
        while len(stack) > 0:
            backtrace += stack.pop()
        return dist, backtrace
    else:
        return dist


print(distance("lanugage", "language", Damerau=True))

(1, 'MMMTMMM')


### Проверка на задачите 1-3 преку тест примерите

In [66]:
test_cases = [('intention', 'execution', 8, 8, 'DSSMISMMMM', 'DSSMISMMMM'),
              ('flowers', 'lfowers', 2, 1, 'DMIMMMMM', 'TMMMMM'),
              ('as', 'asdf', 2, 2, 'MMII', 'MMII'),
              ('effect', 'beauty', 8, 8, 'IMDDSSMI', 'IMDDSSMI'),
              ('the', 'teh', 2, 1, 'MDMI', 'MT'),
              ('challenge', 'challenger', 1, 1, 'MMMMMMMMMI', 'MMMMMMMMMI'),
              ('tree', 'three', 1, 1, 'MIMMM', 'MIMMM'),
              ('accross', 'acres', 4, 4, 'MDMMDSM', 'MDMMDSM')]

for word1, word2, dist1, dist2, back1, back2 in test_cases:
    mydist1, backtrace1 = distance(word1, word2, False)
    print('Levenstein words:', word1, word2, 'Distances:', dist1, mydist1, 'Correct:', dist1 == mydist1,
          'Backtraces:', back1, backtrace1, back1 == backtrace1)
    mydist2, backtrace2 = distance(word1, word2, True)
    print('Damerau-Levenstein:', word1, word2, 'Distances:', dist2, mydist2, 'Correct:', dist2 == mydist2,
          'Backtraces:', back2, backtrace2, back2 == backtrace2)

Levenstein words: intention execution Distances: 8 8 Correct: True Backtraces: DSSMISMMMM DSSMISMMMM True
Damerau-Levenstein: intention execution Distances: 8 8 Correct: True Backtraces: DSSMISMMMM DSSMISMMMM True
Levenstein words: flowers lfowers Distances: 2 2 Correct: True Backtraces: DMIMMMMM DMIMMMMM True
Damerau-Levenstein: flowers lfowers Distances: 1 1 Correct: True Backtraces: TMMMMM TMMMMM True
Levenstein words: as asdf Distances: 2 2 Correct: True Backtraces: MMII MMII True
Damerau-Levenstein: as asdf Distances: 2 2 Correct: True Backtraces: MMII MMII True
Levenstein words: effect beauty Distances: 8 8 Correct: True Backtraces: IMDDSSMI IMDDSSMI True
Damerau-Levenstein: effect beauty Distances: 8 8 Correct: True Backtraces: IMDDSSMI IMDDSSMI True
Levenstein words: the teh Distances: 2 2 Correct: True Backtraces: MDMI MDMI True
Damerau-Levenstein: the teh Distances: 1 1 Correct: True Backtraces: MT MT True
Levenstein words: challenge challenger Distances: 1 1 Correct: True Ba

### Задача 4

In [67]:
sample_text = """Natural language processing is a field of computer science, artificial intelligence, 
and computational linguistics concerned with the interactions between computers and human (natural) languages. As 
such, NLP is related to the area of human–computer interaction. Many challenges in NLP involve: natural language 
understanding, enabling computers to derive meaning from human or natural language input; and others involve natural 
language generation. 


An automated online assistant providing customer service on a web page, an example of an application where natural 
language processing is a major component.[1] Contents  [hide] 1	History 2	Using machine learning 3	Major tasks 
4	Statistical 5	Evaluation 5.1	Objectives 5.2	Timeline of evaluation 5.3	Different types of evaluation 6	
Standardization 7	See also 8	References 9	Further reading History[edit] Main article: History of natural 
language processing The history of NLP generally starts in the 1950s, although work can be found from earlier 
periods. In 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what 
is now called the Turing test as a criterion of intelligence. 

The Georgetown experiment in 1954 involved fully automatic translation of more than sixty Russian sentences into 
English. The authors claimed that within three or five years, machine translation would be a solved problem.[2] 
However, real progress was much slower, and after the ALPAC report in 1966, which found that ten-year-long research 
had failed to fulfill the expectations, funding for machine translation was dramatically reduced. Little further 
research in machine translation was conducted until the late 1980s, when the first statistical machine translation 
systems were developed. 

Some notably successful NLP systems developed in the 1960s were SHRDLU, a natural language system working in 
restricted "blocks worlds" with restricted vocabularies, and ELIZA, a simulation of a Rogerian psychotherapist, 
written by Joseph Weizenbaum between 1964 and 1966. Using almost no information about human thought or emotion, 
ELIZA sometimes provided a startlingly human-like interaction. When the "patient" exceeded the very small knowledge 
base, ELIZA might provide a generic response, for example, responding to "My head hurts" with "Why do you say your 
head hurts?". 

During the 1970s many programmers began to write "conceptual ontologies", which structured real-world information 
into computer-understandable data. Examples are MARGIE (Schank, 1975), SAM (Cullingford, 1978), PAM (Wilensky, 1978), 
TaleSpin (Meehan, 1976), QUALM (Lehnert, 1977), Politics (Carbonell, 1979), and Plot Units (Lehnert 1981). During 
this time, many chatterbots were written including PARRY, Racter, and Jabberwacky. 

Up to the 1980s, most NLP systems were based on complex sets of hand-written rules. Starting in the late 1980s, 
however, there was a revolution in NLP with the introduction of machine learning algorithms for language processing. 
This was due to both the steady increase in computational power (see Moore's Law) and the gradual lessening of the 
dominance of Chomskyan theories of linguistics (e.g. transformational grammar), whose theoretical underpinnings 
discouraged the sort of corpus linguistics that underlies the machine-learning approach to language processing.[3] 
Some of the earliest-used machine learning algorithms, such as decision trees, produced systems of hard if-then rules 
similar to existing hand-written rules. However, part-of-speech tagging introduced the use of hidden Markov models to 
NLP, and increasingly, research has focused on statistical models, which make soft, probabilistic decisions based on 
attaching real-valued weights to the features making up the input data. The cache language models upon which many 
speech recognition systems now rely are examples of such statistical models. Such models are generally more robust 
when given unfamiliar input, especially input that contains errors (as is very common for real-world data), 
and produce more reliable results when integrated into a larger system comprising multiple subtasks. 

Many of the notable early successes occurred in the field of machine translation, due especially to work at IBM 
Research, where successively more complicated statistical models were developed. These systems were able to take 
advantage of existing multilingual textual corpora that had been produced by the Parliament of Canada and the 
European Union as a result of laws calling for the translation of all governmental proceedings into all official 
languages of the corresponding systems of government. However, most other systems depended on corpora specifically 
developed for the tasks implemented by these systems, which was (and often continues to be) a major limitation in the 
success of these systems. As a result, a great deal of research has gone into methods of more effectively learning 
from limited amounts of data. 

Recent research has increasingly focused on unsupervised and semi-supervised learning algorithms. Such algorithms are 
able to learn from data that has not been hand-annotated with the desired answers, or using a combination of 
annotated and non-annotated data. Generally, this task is much more difficult than supervised learning, and typically 
produces less accurate results for a given amount of input data. However, there is an enormous amount of 
non-annotated data available (including, among other things, the entire content of the World Wide Web), 
which can often make up for the inferior results. 

Using machine learning[edit]

This section does not cite any sources. Please help improve this section by adding citations to reliable sources. 
Unsourced material may be challenged and removed. (February 2013) (Learn how and when to remove this template 
message) Modern NLP algorithms are based on machine learning, especially statistical machine learning. The paradigm 
of machine learning is different from that of most prior attempts at language processing. Prior implementations of 
language-processing tasks typically involved the direct hand coding of large sets of rules. The machine-learning 
paradigm calls instead for using general learning algorithms — often, although not always, grounded in statistical 
inference — to automatically learn such rules through the analysis of large corpora of typical real-world examples. A 
corpus (plural, "corpora") is a set of documents (or sometimes, individual sentences) that have been hand-annotated 
with the correct values to be learned. 

Many different classes of machine learning algorithms have been applied to NLP tasks. These algorithms take as input 
a large set of "features" that are generated from the input data. Some of the earliest-used algorithms, 
such as decision trees, produced systems of hard if-then rules similar to the systems of hand-written rules that were 
then common. Increasingly, however, research has focused on statistical models, which make soft, probabilistic 
decisions based on attaching real-valued weights to each input feature. Such models have the advantage that they can 
express the relative certainty of many different possible answers rather than only one, producing more reliable 
results when such a model is included as a component of a larger system. 

Systems based on machine-learning algorithms have many advantages over hand-produced rules:

The learning procedures used during machine learning automatically focus on the most common cases, whereas when 
writing rules by hand it is often not at all obvious where the effort should be directed. Automatic learning 
procedures can make use of statistical inference algorithms to produce models that are robust to unfamiliar input (
e.g. containing words or structures that have not been seen before) and to erroneous input (e.g. with misspelled 
words or words accidentally omitted). Generally, handling such input gracefully with hand-written rules — or more 
generally, creating systems of hand-written rules that make soft decisions — is extremely difficult, error-prone and 
time-consuming. Systems based on automatically learning the rules can be made more accurate simply by supplying more 
input data. However, systems based on hand-written rules can only be made more accurate by increasing the complexity 
of the rules, which is a much more difficult task. In particular, there is a limit to the complexity of systems based 
on hand-crafted rules, beyond which the systems become more and more unmanageable. However, creating more data to 
input to machine-learning systems simply requires a corresponding increase in the number of man-hours worked, 
generally without significant increases in the complexity of the annotation process. The subfield of NLP devoted to 
learning approaches is known as natural language learning (NLL) and its conference CoNLL[4] and peak body SIGNLL[5] 
are sponsored by ACL, recognizing also their links with computational linguistics and Language Acquisition. When the 
aim of computational language learning research is to understand more about human language acquisition, 
or psycholinguistics, NLL overlaps into the related field of computational psycholinguistics. 

Major tasks[edit] The following is a list of some of the most commonly researched tasks in NLP. Note that some of 
these tasks have direct real-world applications, while others more commonly serve as subtasks that are used to aid in 
solving larger tasks. What distinguishes these tasks from other potential and actual NLP tasks is not only the volume 
of research devoted to them but the fact that for each one there is typically a well-defined problem setting, 
a standard metric for evaluating the task, standard corpora on which the task can be evaluated, and competitions 
devoted to the specific task. 

Automatic summarization Produce a readable summary of a chunk of text. Often used to provide summaries of text of a 
known type, such as articles in the financial section of a newspaper. Coreference resolution Given a sentence or 
larger chunk of text, determine which words ("mentions") refer to the same objects ("entities"). Anaphora resolution 
is a specific example of this task, and is specifically concerned with matching up pronouns with the nouns or names 
that they refer to. The more general task of coreference resolution also includes identifying so-called "bridging 
relationships" involving referring expressions. For example, in a sentence such as "He entered John's house through 
the front door", "the front door" is a referring expression and the bridging relationship to be identified is the 
fact that the door being referred to is the front door of John's house (rather than of some other structure that 
might also be referred to). Discourse analysis This rubric includes a number of related tasks. One task is 
identifying the discourse structure of connected text, i.e. the nature of the discourse relationships between 
sentences (e.g. elaboration, explanation, contrast). Another possible task is recognizing and classifying the speech 
acts in a chunk of text (e.g. yes-no question, content question, statement, assertion, etc.). Machine translation 
Automatically translate text from one human language to another. This is one of the most difficult problems, 
and is a member of a class of problems colloquially termed "AI-complete", i.e. requiring all of the different types 
of knowledge that humans possess (grammar, semantics, facts about the real world, etc.) in order to solve properly. 
Morphological segmentation Separate words into individual morphemes and identify the class of the morphemes. The 
difficulty of this task depends greatly on the complexity of the morphology (i.e. the structure of words) of the 
language being considered. English has fairly simple morphology, especially inflectional morphology, and thus it is 
often possible to ignore this task entirely and simply model all possible forms of a word (e.g. "open, opens, opened, 
opening") as separate words. In languages such as Turkish or Manipuri,[6] a highly agglutinated Indian language, 
however, such an approach is not possible, as each dictionary entry has thousands of possible word forms. Named 
entity recognition (NER) Given a stream of text, determine which items in the text map to proper names, 
such as people or places, and what the type of each such name is (e.g. person, location, organization). Note that, 
although capitalization can aid in recognizing named entities in languages such as English, this information cannot 
aid in determining the type of named entity, and in any case is often inaccurate or insufficient. For example, 
the first word of a sentence is also capitalized, and named entities often span several words, only some of which are 
capitalized. Furthermore, many other languages in non-Western scripts (e.g. Chinese or Arabic) do not have any 
capitalization at all, and even languages with capitalization may not consistently use it to distinguish names. For 
example, German capitalizes all nouns, regardless of whether they refer to names, and French and Spanish do not 
capitalize names that serve as adjectives. Natural language generation Convert information from computer databases or 
semantic intents into readable human language. Natural language understanding Convert chunks of text into more formal 
representations such as first-order logic structures that are easier for computer programs to manipulate. Natural 
language understanding involves the identification of the intended semantic from the multiple possible semantics 
which can be derived from a natural language expression which usually takes the form of organized notations of 
natural languages concepts. Introduction and creation of language metamodel and ontology are efficient however 
empirical solutions. An explicit formalization of natural languages semantics without confusions with implicit 
assumptions such as closed-world assumption (CWA) vs. open-world assumption, or subjective Yes/No vs. objective 
True/False is expected for the construction of a basis of semantics formalization.[7] Optical character recognition (
OCR) Given an image representing printed text, determine the corresponding text. Part-of-speech tagging Given a 
sentence, determine the part of speech for each word. Many words, especially common ones, can serve as multiple parts 
of speech. For example, "book" can be a noun ("the book on the table") or verb ("to book a flight"); "set" can be a 
noun, verb or adjective; and "out" can be any of at least five different parts of speech. Some languages have more 
such ambiguity than others. Languages with little inflectional morphology, such as English are particularly prone to 
such ambiguity. Chinese is prone to such ambiguity because it is a tonal language during verbalization. Such 
inflection is not readily conveyed via the entities employed within the orthography to convey intended meaning. 
Parsing Determine the parse tree (grammatical analysis) of a given sentence. The grammar for natural languages is 
ambiguous and typical sentences have multiple possible analyses. In fact, perhaps surprisingly, for a typical 
sentence there may be thousands of potential parses (most of which will seem completely nonsensical to a human). 
Question answering Given a human-language question, determine its answer. Typical questions have a specific right 
answer (such as "What is the capital of Canada?"), but sometimes open-ended questions are also considered (such as 
"What is the meaning of life?"). Recent works have looked at even more complex questions.[8] Relationship extraction 
Given a chunk of text, identify the relationships among named entities (e.g. who is married to whom). Sentence 
breaking (also known as sentence boundary disambiguation) Given a chunk of text, find the sentence boundaries. 
Sentence boundaries are often marked by periods or other punctuation marks, but these same characters can serve other 
purposes (e.g. marking abbreviations). Sentiment analysis Extract subjective information usually from a set of 
documents, often using online reviews to determine "polarity" about specific objects. It is especially useful for 
identifying trends of public opinion in the social media, for the purpose of marketing. Speech recognition Given a 
sound clip of a person or people speaking, determine the textual representation of the speech. This is the opposite 
of text to speech and is one of the extremely difficult problems colloquially termed "AI-complete" (see above). In 
natural speech there are hardly any pauses between successive words, and thus speech segmentation is a necessary 
subtask of speech recognition (see below). Note also that in most spoken languages, the sounds representing 
successive letters blend into each other in a process termed coarticulation, so the conversion of the analog signal 
to discrete characters can be a very difficult process. Speech segmentation Given a sound clip of a person or people 
speaking, separate it into words. A subtask of speech recognition and typically grouped with it. Topic segmentation 
and recognition Given a chunk of text, separate it into segments each of which is devoted to a topic, and identify 
the topic of the segment. Word segmentation Separate a chunk of continuous text into separate words. For a language 
like English, this is fairly trivial, since words are usually separated by spaces. However, some written languages 
like Chinese, Japanese and Thai do not mark word boundaries in such a fashion, and in those languages text 
segmentation is a significant task requiring knowledge of the vocabulary and morphology of words in the language. 
Word sense disambiguation Many words have more than one meaning; we have to select the meaning which makes the most 
sense in context. For this problem, we are typically given a list of words and associated word senses, e.g. from a 
dictionary or from an online resource such as WordNet. In some cases, sets of related tasks are grouped into 
subfields of NLP that are often considered separately from NLP as a whole. Examples include: 

Information retrieval (IR) This is concerned with storing, searching and retrieving information. It is a separate 
field within computer science (closer to databases), but IR relies on some NLP methods (for example, stemming). Some 
current research and applications seek to bridge the gap between IR and NLP. Information extraction (IE) This is 
concerned in general with the extraction of semantic information from text. This covers tasks such as named entity 
recognition, Coreference resolution, relationship extraction, etc. Speech processing This covers speech recognition, 
text-to-speech and related tasks. Other tasks include: 

Native-language identification Stemming Text simplification Text-to-speech Text-proofing Natural language search 
Query expansion Automated essay scoring Truecasing Statistical[edit] Main article: Stochastic grammar Statistical 
natural-language processing uses stochastic, probabilistic, and statistical methods to resolve some of the 
difficulties discussed above, especially those which arise because longer sentences are highly ambiguous when 
processed with realistic grammars, yielding thousands or millions of possible analyses. Methods for disambiguation 
often involve the use of corpora and Markov models. The ESPRIT Project P26 (1984 - 1988), led by CSELT, explored the 
problem of speech recognition comparing knowledge-based approach and statistical ones: the chosen result was a 
completely statistical model.[9] One among the first models of statistical natural language understanding was 
introduced in 1991 by Roberto Pieraccini, Esther Levin, and Chin-Hui Lee from Bell Laboratories.[10] NLP comprises 
all quantitative approaches to automated language processing, including probabilistic modeling, information theory, 
and linear algebra.[11] The technology for statistical NLP comes mainly from machine learning and data mining, 
both of which are fields of artificial intelligence that involve learning from data. 

Evaluation[edit] Objectives[edit] The goal of NLP evaluation is to measure one or more qualities of an algorithm or a 
system, in order to determine: whether the algorithm answers the goals of its designers, or if the system meets the 
needs of its users. Research in NLP evaluation has received considerable attention, because the definition of proper 
evaluation criteria is one way to specify precisely an NLP problem. The metric of NLP evaluation on an algorithmic 
system allows for the integration of language understanding and language generation. A precise set of evaluation 
criteria, which include mainly evaluation data and evaluation metrics can enable several teams to compare their 
solutions for a given NLP problem. 

Timeline of evaluation[edit] In 1983, start of Esprit P26 Project, that evaluated Speech Technologies (including 
general topic such as Syntactic & Semantic Parsing, etc.) comparing rule-based towards statistical approaches.[12] In 
1987, the first evaluation campaign on written texts seems to be a campaign dedicated to message understanding (
Pallet 1998). The Parseval/GEIG project compared phrase-structure grammars (Black 1991). There were series of 
campaigns within Tipster project on tasks like summarization, translation, and searching (Hirschman 1998). In 1994, 
in Germany, the Morpholympics compared German morphological taggers. The Senseval & Romanseval campaigns were 
conducted with the objectives of semantic disambiguation. In 1996, the Sparkle campaign compared syntactic parsers in 
four different languages (English, French, German and Italian). In France, the Grace project compared a set of 21 
taggers for French in 1997 (Adda 1999). In 2004, during the Technolangue/Easy project, 13 parsers for French were 
compared. Large-scale evaluation of dependency parsers were performed in the context of the CoNLL shared tasks in 
2006 and 2007. In France, within the ANR-Passage project (end of 2007), 10 parsers for French were compared – passage 
web site. In Italy, the EVALITA campaign was conducted in 2007,[13] 2009, 2011, and 2014[14] to compare various NLP 
and speech tools for Italian – EVALITA web site. Different types of evaluation[edit] Depending on the evaluation 
procedures, a number of distinctions are traditionally made in NLP evaluation. 

Intrinsic v. extrinsic evaluation Intrinsic evaluation considers an isolated NLP system and characterizes its 
performance with respect to a gold standard result as defined by the evaluators. Extrinsic evaluation, also called 
evaluation in use, considers the NLP system in a more complex setting as either an embedded system or a precise 
function for a human user. The extrinsic performance of the system is then characterized in terms of utility with 
respect to the overall task of the extraneous system or the human user. For example, consider a syntactic parser 
which is based on the output of some part of speech (POS) tagger. An intrinsic evaluation would run the POS tagger on 
structured data, and compare the system output of the POS tagger to the gold standard output. An extrinsic evaluation 
would run the parser with some other POS tagger, and then with the novel POS tagger, and compare the parsing 
accuracy. Black-box v. glass-box evaluation Black-box evaluation requires someone to run an NLP system on a sample 
data set and to measure a number of parameters related to: the quality of the process, such as speed, reliability, 
resource consumption; and most importantly, the quality of the result, such as the accuracy of data annotation or the 
fidelity of a translation. Glass-box evaluation looks at the: design of the system; the algorithms that are 
implemented; the linguistic resources it uses, like vocabulary size or expression set cardinality. Given the 
complexity of NLP problems, it is often difficult to predict performance only on the basis of glass-box evaluation; 
but this type of evaluation is more informative with respect to error analysis or future developments of a system. 
Automatic v. manual evaluation In many cases, automatic procedures can be defined to evaluate an NLP system by 
comparing its output with the gold standard one. Although the cost of reproducing the gold standard can be quite 
high, bootstrapping automatic evaluation on the same input data can be repeated as often as needed without inordinate 
additional costs. However, for many NLP problems the precise definition of a gold standard is a complex task and it 
can prove impossible when inter-annotator agreement is insufficient. Manual evaluation is best performed by human 
judges instructed to estimate the quality of a system, or most often of a sample of its output, based on a number of 
criteria. Although, thanks to their linguistic competence, human judges can be considered as the reference for a 
number of language processing tasks, there is also considerable variation across their ratings. That is why automatic 
evaluation is sometimes referred to as objective evaluation while the human evaluation is perspective. 
Standardization[edit] An ISO subcommittee is working in order to ease interoperability between lexical resources and 
NLP programs. The subcommittee is part of ISO/TC37 and is called ISO/TC37/SC4. Some ISO standards are already 
published but most of them are under construction, mainly on lexicon representation (see LMF), annotation, 
and data category registry. 

Which one has a typo: dor, dor, doors?
In order the system to work, we need to create a good design.

"""

#### Извлекување на зборовите од текстот

In [68]:
def get_words(sample_text, distinct=True):
    sample_text = sample_text.lower()  # to lowercase
    # TODO: remove all non-word characters
    sample_text = re.sub('\[edit\]', '', sample_text)
    sample_text = re.sub('[^\s\-\w]', '', sample_text)
    # TODO: remove all extra white spaces, tabs, new lines, etc.
    sample_text = re.sub('[\-\n\t]', ' ', sample_text)
    sample_text = re.sub('\ds?', ' ', sample_text)
    sample_text = re.sub(r'(\s)\s*', r'\1', sample_text)
    all_words = sample_text.split(" ")
    all_words = [word for word in all_words if len(word) > 1]
    if distinct:
        distinct_words = set(all_words)
        return distinct_words
    else:
        return all_words


print(get_words(sample_text))

{'make', 'great', 'entirely', 'esprit', 'ai', 'another', 'up', 'typically', 'systems', 'tagging', 'devoted', 'calling', 'programs', 'hui', 'corpora', 'there', 'articles', 'purpose', 'aid', 'even', 'furthermore', 'note', 'responding', 'between', 'judges', 'before', 'considered', 'ignore', 'fashion', 'mark', 'verbalization', 'map', 'sometimes', 'determining', 'error', 'databases', 'fidelity', 'correct', 'employed', 'enable', 'omitted', 'similar', 'black', 'but', 'tree', 'repeated', 'structures', 'category', 'italian', 'involves', 'shared', 'connected', 'semantic', 'isolated', 'entity', 'turing', 'accurate', 'coreference', 'performed', 'power', 'series', 'bootstrapping', 'focused', 'optical', 'vocabularies', 'parses', 'desired', 'prior', 'fulfill', 'it', 'basis', 'greatly', 'definition', 'politics', 'conducted', 'which', 'involve', 'its', 'already', 'discourse', 'cullingford', 'especially', 'modern', 'artificial', 'interaction', 'qualm', 'find', 'essay', 'chunks', 'small', 'anr', 'while',

#### Градење на heap дрво на зборови

In [69]:
class WordHeap:
    def __init__(self, words, compare_word):
        self.words = list(words)
        self.compare_word = compare_word
        self.distances = {}

    def get_left_child(self, node_index):
        return 2 * (node_index + 1) - 1

    def get_right_child(self, node_index):
        return 2 * (node_index + 1)

    def get_parent(self, node_index):
        return int(np.floor((node_index - 1) / 2))

    def compare_words(self, node_1, node_2):
        word_1 = self.words[node_1]
        word_2 = self.words[node_2]
        if word_1 not in self.distances.keys():
            dist_1 = distance(
                word_1, self.compare_word, Damerau=True, Backtrace=False)
            self.distances.setdefault(word_1, dist_1)
        else:
            dist_1 = self.distances[word_1]
        if word_2 not in self.distances.keys():
            dist_2 = distance(
                word_2, self.compare_word, Damerau=True, Backtrace=False)
            self.distances.setdefault(word_2, dist_2)
        else:
            dist_2 = self.distances[word_2]
        if dist_1 < dist_2:
            return -1
        elif dist_1 > dist_2:
            return 1
        else:
            if word_1 < word_2:
                return -1
            elif word_1 > word_2:
                return 1
            else:
                return 0

    def adjust_up(self, node):
        while node != 0:
            parent = self.get_parent(node)
            if self.compare_words(node, parent) < 0:
                self.words[node], self.words[parent] = self.words[
                    parent], self.words[node]
            node = parent

    def build_heap(self):
        n = len(self.words)
        for i in range(0, n):
            self.adjust_up(i)

    def adjust_down(self):
        node = 0
        while True:
            left = self.get_left_child(node)
            right = self.get_right_child(node)
            if left > len(self.words) - 1:
                break
            elif right > len(self.words) - 1:
                self.words[node], self.words[left] = self.words[
                    left], self.words[node]
                node = left
            else:
                if self.compare_words(left, right) < 0:
                    self.words[node], self.words[left] = self.words[
                        left], self.words[node]
                    node = left
                else:
                    self.words[node], self.words[right] = self.words[
                        right], self.words[node]
                    node = right

    def return_min(self):
        minimum = self.words[0]
        self.words[0] = "*" * len(self.words)
        self.adjust_down()
        return minimum


all_words = get_words(sample_text)
word_heap = WordHeap(all_words, "make")
word_heap.build_heap()
print(word_heap.return_min())
print(word_heap.return_min())
print(word_heap.return_min())

make
makes
made


#### Три пати се отстранува најблискиот збор од дрвото

In [70]:
def calc_all_distances(word, all_words, num_closest=3):
    word_heap = WordHeap(all_words, word)
    word_heap.build_heap()
    distances = []
    for i in range(0,num_closest):
        best_word = word_heap.return_min()
        dist = distance(word, best_word, Damerau=True, Backtrace=True)
        distances.append((dist[0], best_word, dist[1]))
    return distances

### Проверка на задачa 4 преку тест примерите

In [71]:
all_words = get_words(sample_text)

from collections import OrderedDict
test_cases4 = OrderedDict([
    ('cascade', [(3, 'case', 'MMMDDDM'), (4, 'cache', 'MMDMDSM'), (4, 'cases', 'MMMDDDMI')]),
    ('automotive', [(5, 'automated', 'MMMMMSMDDMI'), (5, 'automatic', 'MMMMMSMMDS'), (6, 'native', 'IMDDDDDMMMM')]),
    ('definition', [(0, 'definition', 'MMMMMMMMMM'), (6, 'decision', 'MMDDSMSMMM'), (6, 'function', 'DDMSMSMMMM')]),
    ('systems', [(0, 'systems', 'MMMMMMM'), (1, 'system', 'MMMMMMD'), (4, 'items', 'DDSMMMM')]),
    ('dataset', [(3, 'data', 'MMMMDDD'), (4, 'databases', 'MMMIIMMMS'), (4, 'set', 'DDDDMMM')]),
    ('extrinsic', [(0, 'extrinsic', 'MMMMMMMMM'), (4, 'intrinsic', 'SSMMMMMMM'), (5, 'metric', 'IMDMMDDDMM')]),
    ('lexicon', [(0, 'lexicon', 'MMMMMMM'), (4, 'levin', 'MMSMDDM'), (4, 'lexical', 'MMMMMSS')]),
    ('analysis', [(3, 'analyses', 'MMMMDTIM'), (3, 'analysis', 'MMMMDTIM'), (5, 'any', 'MMDDDDMD')]),
    ('referee', [(2, 'refer', 'MMMMMDD'), (2, 'reference', 'MMMMMMIIM'), (3, 'references', 'MMMMMMIIMI')])])

for word1, ground_distances in test_cases4.items():
    distances = calc_all_distances(word1, all_words)
    print(distances[0], ground_distances[0])
    print(distances[1], ground_distances[1])
    print(distances[2], ground_distances[2])

(3, 'case', 'MMMDDDM') (3, 'case', 'MMMDDDM')
(4, 'cache', 'MMDMDSM') (4, 'cache', 'MMDMDSM')
(4, 'cases', 'MMMDDDMI') (4, 'cases', 'MMMDDDMI')
(5, 'automated', 'MMMMMSMDDMI') (5, 'automated', 'MMMMMSMDDMI')
(5, 'automatic', 'MMMMMSMMDS') (5, 'automatic', 'MMMMMSMMDS')
(6, 'native', 'IMDDDDDMMMM') (6, 'native', 'IMDDDDDMMMM')
(0, 'definition', 'MMMMMMMMMM') (0, 'definition', 'MMMMMMMMMM')
(6, 'decision', 'MMDDSMSMMM') (6, 'decision', 'MMDDSMSMMM')
(6, 'function', 'DDMSMSMMMM') (6, 'function', 'DDMSMSMMMM')
(0, 'systems', 'MMMMMMM') (0, 'systems', 'MMMMMMM')
(1, 'system', 'MMMMMMD') (1, 'system', 'MMMMMMD')
(4, 'items', 'DDSMMMM') (4, 'items', 'DDSMMMM')
(3, 'data', 'MMMMDDD') (3, 'data', 'MMMMDDD')
(4, 'databases', 'MMMIIMMMS') (4, 'databases', 'MMMIIMMMS')
(4, 'set', 'DDDDMMM') (4, 'set', 'DDDDMMM')
(0, 'extrinsic', 'MMMMMMMMM') (0, 'extrinsic', 'MMMMMMMMM')
(4, 'intrinsic', 'SSMMMMMMM') (4, 'intrinsic', 'SSMMMMMMM')
(5, 'metric', 'IMDMMDDDMM') (5, 'metric', 'IMDMMDDDMM')
(0, 'lexicon

### Задача 5

In [72]:
def calculate_occurrances(corpus):
    corpus = get_words(corpus, distinct = False)
    occurances = {
    }  # dict with key the word or tuple with the pair of words in a bigram, value a list with 2 values: the number of occurances, the probability of occurence
    n = len(corpus)
    for i in range(0, n - 1):
        word = corpus[i].lower()
        if len(word) > 2:
            occurances.setdefault(word, [0, 0])
            occurances[word][0] += 1
            occurances[word][1] += 1 / n
        word_2 = corpus[i + 1].lower()
        bigram = (word, word_2)
        occurances.setdefault((word, word_2), [0, 0])
        occurances[bigram][0] += 1
        occurances[bigram][1] += 1 / (n - 1)
    word = corpus[n - 1].lower()
    if len(word) > 2:
        occurances.setdefault(word, [0, 0])
        occurances[word][0] += 1
        occurances[word][1] += 1 / n
    return occurances

### Проверка на задачa 5 преку тест примерите

In [73]:
test_cases5 = ['between lexical', 'objectives', 'campaign', 'order to', 'searching', 'into separate']
occurances = calculate_occurrances(sample_text)
for tc in test_cases5:
    # TODO: calculate these
    is_bigram = None
    is_bigram = len(tc.split(' ')) == 2
    num_occurances = None
    prob = None
    if is_bigram:
        bigram = tc.split(' ')
        bigram[0] = bigram[0].lower()
        bigram[1] = bigram[1].lower()
        num_occurances = occurances[tuple(bigram)][0]
        prob = occurances[tuple(bigram)][1]
    else:
        num_occurances = occurances[tc.lower()][0]
        prob = occurances[tc.lower()][1]
    print(tc, is_bigram, num_occurances, prob)

between lexical True 1 0.0002690341673392521
objectives False 3 0.0008068854222700375
campaign False 4 0.0010758472296933835
order to True 3 0.0008071025020177562
searching False 2 0.0005379236148466917
into separate True 1 0.0002690341673392521


### Задача 6

#### Предложување на збор само според растојанието

In [74]:
def propose_damerau(word, all_words):
    if word in all_words: # if real word spelling error
        all_words.remove(word)
    d = calc_all_distances(word, all_words, num_closest=1)
    # TODO: clean the output
    return d[0][1]

#### Предложување на збор според растојанието и веројатноста

In [75]:
def propose_damerau_prob(word1, word2, corpus):
    # TODO: if word1 precedes word2, propose a substitution for word2, occurances is obtained with calculate_occurrances
    occurances = calculate_occurrances(corpus)
    all_words = get_words(corpus)
    if word2 in all_words: # if real word spelling error
        all_words.remove(word2)
    d = calc_all_distances(word2, all_words, num_closest=10)
    best_word = d[0]
    for i in range(1, 10):
        if d[i][0] < best_word[0]:
            best_word = d[i]
        elif d[i][0] == best_word[0]:
            if (word1, d[i][1]
                ) in occurances.keys() and word1 in occurances.keys():
                prob = occurances[(word1, d[i][1])][1] / occurances[word1][1] #uslovna verojatnost
            else:
                prob = 1 / (2 * len(all_words)) # namesto 0
            if (word1, best_word[1]
                ) in occurances.keys() and word1 in occurances.keys():
                best_prob = float(occurances[
                    (word1, best_word[1])][1]) / occurances[word1][1] #uslovna verojatnost
            else:
                best_prob = 1 / (2 * len(all_words)) # namesto 0
            if prob >= best_prob:
                best_word = d[i]
    return best_word[1]

In [76]:
test_cases6 = [
    'order tu', 'speech recogniton', 'parsers gor', 'computers an', 'to wort'
]
for tc in test_cases6:
    w1, w2 = tc.split(' ')
    print('Words:', tc, 'Proposed Damerau:', propose_damerau(w2, all_words),
          'Proposed Damerau+Prob:', propose_damerau_prob(w1, w2, sample_text))

Words: order tu Proposed Damerau: at Proposed Damerau+Prob: to
Words: speech recogniton Proposed Damerau: recognition Proposed Damerau+Prob: recognition
Words: parsers gor Proposed Damerau: or Proposed Damerau+Prob: or
Words: computers an Proposed Damerau: and Proposed Damerau+Prob: and
Words: to wort Proposed Damerau: or Proposed Damerau+Prob: work
